# BOW & TF-IDF - recitation

``
In this recitation, you will experience with implementing "tf-idf document vectorizer", and see how it can be use to solve practical problem.
``

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from functools import reduce
import operator

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

```What we going to do here is to solve the next problem:
Given a set of articles writen by different authors, we want to decide the "authorship" of a new article writen by one of the authors.```

In [2]:
## can be found in: https://drive.google.com/open?id=1Ni5Rb_q9gdCIGPW2i1oyH2a5G823fOio
df = pd.read_csv('data/Gungor_2018_VictorianAuthorAttribution.csv')

In [3]:
## we have articles of 10 different authors
df.groupby('author').agg(article_count=pd.NamedAgg(column="text", aggfunc="count"))

,article_count
author,
8,6914
14,2696
19,1543
21,2307
26,4441
33,1742
37,2387
39,2266
45,2312


## "bag of words" (BOW)

wikipedia link: https://en.wikipedia.org/wiki/Bag-of-words_model

```Bag of words is a way to represent a document (or a sentance) as the "counts" of the words in it.```

``That way, if we have the vocabulary of the language (all the words in the language) and we agree on some order of thses words (say ["television", "mouse", ...]), we can represent any socument as a vector that each value in it is the count of the corresponding word in the document.``

example:

```doc       = "John likes to watch movies. Mary likes movies too.
vocabulary = ["John, "likes", "to", "watch", "movies", "Mary", "too", "book"].
=> bow = [1, 2, 1, 1, 2, 1, 1, 0].```

With this representation we can measure "distance" between documents.

```The most way to do it is "cosine similaruty". For vector v1, and vector v2, we define "cosine similaruty" as:```

$$similarity(v1, v2) = \frac{<v1, w2>}{\lVert v1 \rVert \cdot \lVert v2 \rVert}$$

```And it is the value of``` $cos(\theta)$, ```where``` $\theta$ ```is the angle between v1 and v2. (big similarity(v1, v2) means v1 and v2 are close to each other)```

(Explanation can be found in: https://en.wikipedia.org/wiki/Cosine_similarity)

``Lets make our articles to be in BOW shape:``

In [4]:
def clean_words(document):
    
    doc_strings = document.split(' ')
    doc_strings = [i.strip() for i in doc_strings]
    doc_strings = [i for i in doc_strings if i != '']
    
    return doc_strings

In [5]:
documents = df['text'].apply(clean_words).tolist()
authors = df['author'].values

``Lets spit the data into train and test. We remember that the meaning of the train and test is: data we have (train), and data we will see in the future (test).``

``Because we have only the train data when we "learn", the words we consider for the model are only the words we see in the train.``

In [6]:
train_docs, test_docs, train_authors, test_authors = train_test_split(documents, authors)

In [7]:
all_words = reduce(operator.iconcat, train_docs, [])
vocabulary = np.array(list(set(all_words))) # the vocabulary composed only from the words in the train

Now we create vectorizer that given the vocabulary, can turn documents to BOW vectors:

In [8]:
def create_document_vectorizer(words):
    
    indexer = {w:i for i,w in enumerate(words)}
    
    def vectorizer(document):
        N, vocab_size = len(documents), len(words)
        vec = np.zeros(vocab_size)
        for w in document:
            if w in indexer: # if we dont have the word in the vocabulary we not counting it
                vec[indexer[w]] += 1
        return vec
    
    return vectorizer

In [9]:
vectorizer = create_document_vectorizer(vocabulary)

In [10]:
train_bow_vectors = np.vstack([vectorizer(doc) for doc in train_docs])
test_bow_vectors = np.vstack([vectorizer(doc) for doc in test_docs])

print("train_bow_vectors", train_bow_vectors.shape)
print("test_bow_vectors", test_bow_vectors.shape)

train_bow_vectors (21324, 10000)
test_bow_vectors (7109, 10000)


In [17]:
def pairwise_cosine_similarity(vecs1, vecs2):
    """
    vecs1: N1 x M dim matrix (N1 vectors with dim M)
    vecs2: N2 x M dim matrix (N2 vectors with dim M)
    
    return: N1 x N2 dim matrix "scores", where scores[i,j] = cosine_similarity(vecs1[i], vecs2[j])
    """
    
    norms_vecs1 = np.linalg.norm(vecs1, axis=1)
    norms_vecs2 = np.linalg.norm(vecs2, axis=1)

    print("norms_vecs1", norms_vecs1.shape)
    print("norms_vecs2", norms_vecs2.shape)

    vecs1_normalized = (vecs1.T / norms_vecs1).T
    vecs2_normalized = (vecs2.T / norms_vecs2).T
    
    print("vecs1_normalized", vecs1_normalized.shape)
    print("vecs2_normalized", vecs2_normalized.shape)
    
    scores = vecs1_normalized @ vecs2_normalized.T
    print("scores", scores.shape)
    
    return scores

```In order to decise the author of every document in the test, we will calculate the cosine similarity between every doc in the test to every doc in the train.
We will say that the author of a document in the test is the author of its "most similar document in the train". (highest score)```

In [18]:
scores = pairwise_cosine_similarity(test_bow_vectors, train_bow_vectors)

norms_vecs1 (7109,)
norms_vecs2 (21324,)
vecs1_normalized (7109, 10000)
vecs2_normalized (21324, 10000)
scores (7109, 21324)


In [19]:
predicted_authors = train_authors[scores.argmax(axis=1)]

In [20]:
print('accuracy: ', np.mean(predicted_authors == test_authors))

accuracy:  0.899845266563511


In [21]:
print('confision matrix:')
confusion_matrix(test_authors, predicted_authors)

confision matrix:


array([[1549,   22,    6,   13,   11,   16,   16,   20,   31,   26],
       [  19,  564,    1,    4,    9,    6,    2,   12,   33,    5],
       [  18,    4,  354,    0,    0,    0,   13,    0,    0,   13],
       [  18,    2,    2,  476,    1,    9,   12,   27,   14,    5],
       [   3,    0,    1,    0, 1103,    0,    1,    0,    0,    0],
       [  12,    2,    1,    3,    1,  401,    3,    9,    8,    9],
       [  32,    3,   14,    8,    3,    5,  497,    3,    7,   15],
       [   8,    1,    0,    9,    0,    0,    1,  570,    9,    3],
       [  25,   12,    5,    5,    8,    8,    9,   12,  496,    6],
       [  16,    6,    2,    3,    2,    2,   14,    5,    8,  387]],
      dtype=int64)

```Not bad hah? Lets see how can we improve it.```

## tf-idf (term frequency–inverse document frequency)

wikipedia link: https://en.wikipedia.org/wiki/Tf%E2%80%93idf

```Main problem of "bag of words" approach (in addition to the lost of "words order"), is that we give significant wight to "stop words.```

<b>stop words:</b> ```are the most common words in the language (i.e. it, I, was, to, ...), and they can be found in almost all documents. Therfore, they cannot help us "distinguish" between documents, and usualy their value in the BOW vector is very high compared to other words.```

```tf-idf come to solve this problem by giving "score" to each word that represent: how much this word "distinguish" the documents we have.```

Say we $N$ documents $\{doc_1, ..., doc_N\}$, and vocabulary $\{w_1, ..., w_M\}$.

We define $n_w$ = "in how many documents w appears".

Lets define the "idf" (inverse document frequency) score for a word $w$ as: $idf(w) = log(\frac{N}{n_w + 1})$

```(low idf value means "stop word", and high as distinguishing word)```


```We now can improve our BOW representation of some new document by multiplying every value in the BOW vector with the corresponting idf score
(td-idf[i] = bow[i]*idf[i]), and get tf-idf representation.```



<b>Remark:</b>

```The training process of this tf-idf "model" is the calculation of the idf scores.
(And there are more ways to calculate idf scores)```

In [24]:
def culc_idf_scores(bow_vectors):
    """
    bow_vectors: matrix of shape (num documents x vocabulary size)
    
    return: idf score for every word in the vocabulary
    """
    
    n_w = np.sum(bow_vectors!=0, axis=0)
    idf = np.log(bow_vectors.shape[0]/(n_w+1))
    
    return idf

In [25]:
idf = culc_idf_scores(train_bow_vectors)

```Lets see which words got low scores:```

In [26]:
vocabulary[np.argsort(idf)[:50]]

array(['a', 'to', 'in', 'the', 'and', 'of', 'for', 'it', 'that', 'with',
       'as', 'on', 'at', 'but', 'not', 'be', 'by', 'all', 'was', 's', 'Γ',
       'he', 'from', 'his', 'this', 'have', 'i', 'no', 'so', 'had', 'is',
       'one', 'an', 'which', 'or', 'there', 'were', 'when', 'they', 'if',
       'been', 'him', 'more', 'what', 'would', 'who', 'out', 'than',
       'into', 'them'], dtype='<U16')

```Lets see which words got high scores:```

In [27]:
vocabulary[np.argsort(idf)[::-1][:50]]

array(['definitely', 'mattered', 'annoy', 'victoria', 'tinted', 'draped',
       'risked', 'ordeal', 'photographs', 'tipped', 'balcony', 'scant',
       'cracks', 'arouse', 'overtook', 'cowardice', 'chasing', 'overcoat',
       'galloped', 'xxx', 'overgrown', 'comments', 'spire', 'links',
       'prefers', 'photograph', 'slips', 'leafy', 'confidences',
       'outbreak', 'splendidly', 'selecting', 'tinged', 'typical',
       'propped', 'laboriously', 'distasteful', 'richness', 'hymns',
       'invest', 'greetings', 'permitting', 'gust', 'occupants', 'oval',
       'willow', 'cope', 'illusions', 'shaven', 'gaudy'], dtype='<U16')

In [28]:
train_tfidf_vectors = train_bow_vectors * idf
test_tfidf_vectors  = test_bow_vectors * idf

In [29]:
scores = pairwise_cosine_similarity(test_tfidf_vectors, train_tfidf_vectors)

norms_vecs1 (7109,)
norms_vecs2 (21324,)
vecs1_normalized (7109, 10000)
vecs2_normalized (21324, 10000)
scores (7109, 21324)


In [30]:
predicted_authors = train_authors[scores.argmax(axis=1)]

In [31]:
print('accuracy: ', np.mean(predicted_authors == test_authors))

accuracy:  0.9518919679279786


In [32]:
print('confision matrix:')
confusion_matrix(test_authors, predicted_authors)

confision matrix:


array([[1663,    4,    7,    1,    7,    7,    4,    7,    6,    4],
       [  19,  611,    0,    0,    8,    2,    2,    3,    3,    7],
       [   2,    0,  381,    4,    0,    0,    5,    1,    1,    8],
       [   6,    0,    6,  530,    0,    6,    2,    9,    7,    0],
       [   0,    0,    0,    0, 1104,    0,    0,    1,    0,    3],
       [   9,   10,    2,    4,    3,  407,    2,    2,    9,    1],
       [   8,    7,    6,    4,    2,    5,  532,    0,   21,    2],
       [   2,    0,    0,    3,    0,    2,    0,  590,    1,    3],
       [  12,    2,    8,    4,    7,    2,    3,    5,  542,    1],
       [   9,    0,    8,    1,    1,    3,    8,    4,    4,  407]],
      dtype=int64)

## Bonus

```Make a vector for every author, as the mean of all his tf-idf vectors in the train.
Try to compare the test tf-idf vectors only to those vectors now instead of all the vectors.
How the accuracy now? how is the computation time?```

In [48]:
list_authors = np.unique(train_authors)
N = train_tfidf_vectors.shape[0]
count_test = 0
list_tfidf_autors = []

for author in list_authors:
    tfidf_author = train_tfidf_vectors[train_authors==author]
    count_test += tfidf_author.shape[0]
    
    mean_tfidf_author = np.mean(tfidf_author, axis=0)
    list_tfidf_autors.append(mean_tfidf_author)

all_mean_tfidf_author = np.stack(list_tfidf_autors)
assert count_test == N

scores = pairwise_cosine_similarity(test_tfidf_vectors, all_mean_tfidf_author)
predicted_authors = list_authors[scores.argmax(axis=1)]

print('accuracy: ', np.mean(predicted_authors == test_authors))
confusion_matrix(test_authors, predicted_authors)

norms_vecs1 (7109,)
norms_vecs2 (10,)
vecs1_normalized (7109, 10000)
vecs2_normalized (10, 10000)
scores (7109, 10)
accuracy:  0.8641159094106062


array([[1491,   39,    3,   24,   17,   27,   42,    6,   42,   19],
       [  39,  507,    2,    2,   13,   17,   15,    1,   22,   37],
       [   2,    0,  276,   16,    0,    0,   97,    0,    4,    7],
       [   6,    3,   14,  464,    0,   20,   40,    9,    5,    5],
       [   0,   11,    0,    0, 1082,    0,    2,    0,    4,    9],
       [   5,    0,    4,    1,    1,  423,    7,    5,    1,    2],
       [   2,    0,   39,    4,    0,    9,  516,    3,   10,    4],
       [  20,    2,    1,   15,    2,    6,    2,  537,   16,    0],
       [   7,   13,    0,    0,    1,    9,   55,    9,  492,    0],
       [  13,    2,    5,    2,    1,    6,   37,    0,   24,  355]],
      dtype=int64)

```Now do the same, but instead of the one "mean" vector for each author, find 3 vectors for each author as the cluster centers of KMeans(k=3) on his train tf-idf vectors. (try low n_init to reduce run time)
Is it more helpful?```

In [50]:
from sklearn.cluster import KMeans

list_authors = np.unique(train_authors)
K = 3
N = train_tfidf_vectors.shape[0]
count_test = 0
list_tfidf_autors = []

for author in list_authors:
    tfidf_author = train_tfidf_vectors[train_authors==author]
    count_test += tfidf_author.shape[0]
    
    kmeans = KMeans(n_clusters=K, n_init=3).fit(tfidf_author)
    clusters = kmeans.cluster_centers_
    list_tfidf_autors += list(clusters)

all_mean_tfidf_author = np.stack(list_tfidf_autors)
list_authors = np.repeat(list_authors, K)
assert count_test == N

print("all_mean_tfidf_author", all_mean_tfidf_author.shape)

scores = pairwise_cosine_similarity(test_tfidf_vectors, all_mean_tfidf_author)
predicted_authors = list_authors[scores.argmax(axis=1)]

print('accuracy: ', np.mean(predicted_authors == test_authors))
confusion_matrix(test_authors, predicted_authors)

all_mean_tfidf_author (30, 10000)
norms_vecs1 (7109,)
norms_vecs2 (30,)
vecs1_normalized (7109, 10000)
vecs2_normalized (30, 10000)
scores (7109, 30)
accuracy:  0.8991419327612885


array([[1513,   52,   15,   12,   39,   12,   19,    4,   17,   27],
       [  20,  594,    1,    3,    2,   12,   14,    1,    8,    0],
       [   1,    0,  325,    0,    0,    0,   55,    0,    7,   14],
       [   9,    6,   31,  459,    0,   12,   21,   14,    9,    5],
       [   4,    1,    0,    0, 1077,    0,    0,    0,    2,   24],
       [   4,    0,    3,    2,    0,  422,    8,    4,    4,    2],
       [   3,    1,   21,    4,    0,   11,  499,    1,   39,    8],
       [  10,    4,    0,   12,    0,    5,    2,  560,    8,    0],
       [   4,    2,    0,    1,    9,    7,    8,    2,  549,    4],
       [  12,    3,   11,    0,    1,    2,   12,    0,   10,  394]],
      dtype=int64)